In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [3]:
from dotenv import load_dotenv
import nest_asyncio
import os

# PREPARE ENVIRONMENT
os.environ["SWE_AGENT_CONFIG_DIR"] = "."
os.environ["SWE_AGENT_TOOLS_DIR"] = "tools"
os.environ["SWE_AGENT_TRAJECTORY_DIR"] = "trajectories"

os.makedirs("replays", exist_ok=True)
os.makedirs("trajectories", exist_ok=True)

load_dotenv()
nest_asyncio.apply()

In [4]:
import asyncio
import modal

# CLEAN UP RUNNING SANDBOXES
sandboxes: list[modal.Sandbox] = []
async for sandbox in modal.Sandbox.list.aio(
    app_id=modal.App.lookup("swe-rex", create_if_missing=True).app_id
):
    sandboxes.append(sandbox)
_ = await asyncio.gather(*[sandbox.terminate.aio() for sandbox in sandboxes])

In [ ]:
import art
from art.local import LocalBackend
from rollout import ModelConfig

# INITIALIZE MODEL
backend = LocalBackend()
model = art.TrainableModel(
    name="002",
    project="sweagent",
    config=ModelConfig(
        max_input_tokens=40_960,
        system_prompt_suffix="\n/no_think",
        xml_function_calling=True,
    ),
    base_model="Qwen/Qwen3-32B",
    _internal_config=art.dev.InternalModelConfig(
        init_args=art.dev.InitArgs(
            max_seq_length=40_960,
        ),
    ),
)
await model.register(backend)

In [ ]:
from instances import as_instances_iter, get_filtered_swe_smith_instances_df
from rollout import rollout

# TRAIN MODEL
instances = (
    get_filtered_swe_smith_instances_df()
    .sample(fraction=1.0, shuffle=True, seed=42)
    .pipe(as_instances_iter)
)

async for trajectory_groups in art.trajectory_group_batches(
    (
        art.TrajectoryGroup(rollout(model, instance) for _ in range(4))
        for instance in instances
    ),
    batch_size=4,
    max_concurrent_batches=2,
    skip_batches=await model.get_step(),
):
    await model.train(
        trajectory_groups,
        _config=art.dev.TrainConfig(
            enable_assistant_message_training=True,
        ),
    )